# Fraud Detection Demo with Howso Engine

Synthetic dataset of mobile money transactions sourced from:<br>
E. A. Lopez-Rojas , A. Elmir, and S. Axelsson. "PaySim: A financial mobile money simulator for fraud detection". In: The 28th European Modeling and Simulation Symposium-EMSS, Larnaca, Cyprus. 2016

In [1]:
import pandas as pd
import numpy as np
from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

In [2]:
df_raw = pd.read_csv("./PS_20174392719_1491204439457_log.csv")

# Select a subsample for use in the notebook environment
df = df_raw.sample(10000, random_state=42)
df = df.reset_index(drop=True)

df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,278,CASH_IN,330218.42,C632336343,20866.00,351084.42,C834976624,452419.57,122201.15,0,0
1,15,PAYMENT,11647.08,C1264712553,30370.00,18722.92,M215391829,0.00,0.00,0,0
2,10,CASH_IN,152264.21,C1746846248,106589.00,258853.21,C1607284477,201303.01,49038.80,0,0
3,403,TRANSFER,1551760.63,C333676753,0.00,0.00,C1564353608,3198359.45,4750120.08,0,0
4,206,CASH_IN,78172.30,C813403091,2921331.58,2999503.88,C1091768874,415821.90,337649.60,0,0


In [3]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.00000,10000.0000
mean,243.990100,1.737107e+05,9.050636e+05,9.272651e+05,1.063047e+06,1.184138e+06,0.00190,0.0001
std,141.704463,4.662073e+05,3.086712e+06,3.122753e+06,2.840653e+06,3.034161e+06,0.04355,0.0100
min,1.000000,1.260000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.0000
25%,156.000000,1.341284e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.0000
50%,249.000000,7.420973e+04,1.409942e+04,0.000000e+00,1.345331e+05,2.118248e+05,0.00000,0.0000
75%,345.250000,2.058379e+05,1.088195e+05,1.573190e+05,9.483795e+05,1.114265e+06,0.00000,0.0000
max,717.000000,1.727101e+07,3.359321e+07,3.374855e+07,9.234964e+07,9.261398e+07,1.00000,1.0000


Flagged transactions are cancelled in this dataset, so columns `oldbalanceOrg`, `newbalanceOrig`, `oldbalanceDest`, `newbalanceDest` should not be used in the analysis.

In [4]:
df = df.drop(['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'], axis=1)
df.head()

,step,type,amount,nameOrig,nameDest,isFraud,isFlaggedFraud
0,278,CASH_IN,330218.42,C632336343,C834976624,0,0
1,15,PAYMENT,11647.08,C1264712553,M215391829,0,0
2,10,CASH_IN,152264.21,C1746846248,C1607284477,0,0
3,403,TRANSFER,1551760.63,C333676753,C1564353608,0,0
4,206,CASH_IN,78172.30,C813403091,C1091768874,0,0


In [5]:
features = infer_feature_attributes(df)
print(features)

{'step': {'type': 'continuous', 'data_type': 'number', 'decimal_places': 0, 'original_type': {'data_type': 'integer', 'size': 8}, 'bounds': {'min': 1.0, 'max': 1097.0, 'allow_null': False}}, 'type': {'type': 'nominal', 'original_type': {'data_type': 'string'}}, 'amount': {'type': 'continuous', 'data_type': 'number', 'decimal_places': 2, 'original_type': {'data_type': 'numeric', 'size': 8}, 'bounds': {'min': 1.0, 'max': 24154952.75, 'allow_null': True}}, 'nameOrig': {'type': 'nominal', 'original_type': {'data_type': 'string'}}, 'nameDest': {'type': 'nominal', 'original_type': {'data_type': 'string'}}, 'isFraud': {'type': 'nominal', 'data_type': 'number', 'decimal_places': 0, 'original_type': {'data_type': 'integer', 'size': 8}}, 'isFlaggedFraud': {'type': 'nominal', 'data_type': 'number', 'decimal_places': 0, 'original_type': {'data_type': 'integer', 'size': 8}}}


In [6]:
# -- step is a measure of time, so we'll correct the inferred feature type

features["step"]["type"] = "ordinal"

In [7]:
action_features = ['isFraud']
context_features = features.get_names(without=action_features)

### Create and Train the Trainee

In [8]:
trainee = Trainee(features=features)

trainee.train(df)

### Analyze (Targeted)

In [9]:
trainee.analyze(context_features=context_features, action_features=action_features)

### Select Test Cases and React

In order to test the predictive capabilities of the Trainee, randomly select fraud cases and ask the Trainee to react.

In [10]:
test = df_raw.sample(300, weights='isFraud', random_state=70)
test.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6342578,692,TRANSFER,46072.33,C1599191169,46072.33,0.00,C1035155877,0.0,0.00,1,0
6281569,649,TRANSFER,150205.64,C1014484059,150205.64,0.00,C1745685493,0.0,0.00,1,0
5996405,425,TRANSFER,10000000.00,C1756917426,29585040.37,19585040.37,C1192036077,0.0,0.00,1,0
6296625,674,CASH_OUT,6250438.78,C951549308,6250438.78,0.00,C1717545219,0.0,6250438.78,1,0
2975525,231,TRANSFER,273821.76,C121294173,273821.76,0.00,C815544341,0.0,0.00,1,0


All test cases are fraud, so it will be straightforward to test the model.

In [11]:
results = trainee.react(
    test[context_features],
    context_features=context_features,
    action_features=action_features,
)

predictions = results['action'][action_features]
print('Accuracy: ',np.sum(predictions, axis=0)/300.)

Accuracy:  isFraud    0.11
dtype: float64


### Evaluate Trainee without Train-Test Split

The train-test split appears to have very poor results, which isn't surprising with such a small percentage of fraudulent transactions, but there are more appropriate statistical approaches to testing our Trainee.

We'll take a different approach with `react_aggregate()` method.

In [15]:
stats = trainee.react_aggregate(
   #action_feature='isFraud',
   action_features=['isFraud'],
   details={
      'prediction_stats': True,
      'selected_prediction_stats': ['accuracy','confusion_matrix','precision','recall']
   }
)
stats

{'confusion_matrix': {'isFraud': {'other_counts': {},
   'leftover_correct': 0,
   'matrix': {'0': {'0': 999}},
   'leftover_incorrect': 0}},
 'recall': {'isFraud': 1},
 'precision': {'isFraud': 1},
 'accuracy': {'isFraud': 1}}

These metrics look a little too good. Anomaly detection may be a better approach.

### Anomaly Detection

We'll apply targetless analysis to see if anomaly detection can predict fraud cases.

In [17]:
from howso.visuals import plot_anomalies

In [18]:
trainee.analyze()

In [21]:
trainee.react_into_features(
    familiarity_conviction_addition=True,
    distance_contribution=True
)

stored_convictions = trainee.get_cases(
    session=trainee.active_session,
    features=[
        'step', 'type', 'amount', 'nameOrig', 'nameDest', 'isFlaggedFraud',
        'familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution']
)

In [23]:
# -- set threshold for anomalous case
convict_threshold = 0.75
low_conv = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold]
low_conv = low_conv.sort_values('familiarity_conviction_addition', ascending=True)

# -- find average distance contribution as point of comparison for outliers
average_dist_contribution = low_conv['distance_contribution'].mean()

cat = [
    'inlier' if d < average_dist_contribution else
    'outlier' for d in low_conv['distance_contribution']
]
low_conv['category'] = cat

In [28]:
outliers = low_conv[low_conv['category'] == 'outlier'].reset_index(drop=True)
outliers.head()

,step,type,amount,nameOrig,nameDest,isFlaggedFraud,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,212,TRANSFER,4953893.08,C728984460,C639921569,1,0.000317,548,daa39dff-e52c-45a3-b00b-3095e2246423,6.425372e+09,outlier
1,621,TRANSFER,2060748.45,C1129518026,C1288050386,0,0.003386,4030,daa39dff-e52c-45a3-b00b-3095e2246423,1.324452e+09,outlier
2,382,CASH_OUT,1595587.46,C1681145385,C1060758741,0,0.004098,4560,daa39dff-e52c-45a3-b00b-3095e2246423,1.207351e+09,outlier
3,194,TRANSFER,1501297.88,C1043170433,C768896772,0,0.004115,7526,daa39dff-e52c-45a3-b00b-3095e2246423,1.205581e+09,outlier
4,15,CASH_OUT,164500.81,C1689700172,C149696600,0,0.004211,9545,daa39dff-e52c-45a3-b00b-3095e2246423,1.192984e+09,outlier


In [34]:
print('Number of outliers: ',len(outliers))
print('Number of fraud cases in train: ',len(df[df['isFraud']==1]))

Number of outliers:  19
Number of fraud cases in train:  19


To follow up, we should ensure that these cases are the same, and if not, why not?